# 7d – Gravity / deterrence model

Estimate a **gravity model** of utilisation: flows ∝ capacity^α / distance^β.  
Here we fit α, β via Poisson regression on historic patient‑flow counts (if available).

In [ ]:

import math, pandas as pd, numpy as np
import statsmodels.api as sm
from pathlib import Path

DATA_DIR = Path('.')
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'
FLOW_CSV  = DATA_DIR / 'historic_flows.csv'    # spoke→acute utilisation (rows=spoke, columns=hub)

acute = pd.read_csv(ACUTE_CSV)
spokes = pd.concat([
    pd.read_csv(CDC_CSV),
    pd.read_csv(CH_CSV)
], ignore_index=True)

R = 6371
def haversine(lat1, lon1, lat2, lon2):
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ, dλ = φ2 - φ1, λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# Build long‑format frame of all spoke‑hub pairs
pairs = []
for _, spk in spokes.iterrows():
    for _, hub in acute.iterrows():
        pairs.append({
            'spoke': spk.Name,
            'hub': hub.Name,
            'distance': haversine(spk.latitude, spk.longitude,
                                  hub.latitude, hub.longitude),
            'capacity': hub.get('capacity', 1)
        })
df = pd.DataFrame(pairs)

# Merge observed flow counts if available
if FLOW_CSV.exists():
    flows = pd.read_csv(FLOW_CSV)
    df = df.merge(flows, on=['spoke', 'hub'], how='left').fillna({'flow': 1})

# Gravity model: ln(flow) ~ ln(capacity) - β * ln(distance)
df['log_cap'] = np.log(df.capacity)
df['log_dist'] = np.log(df.distance + 1e-6)

model = sm.GLM(df['flow'], sm.add_constant(df[['log_cap', 'log_dist']]),
               family=sm.families.Poisson()).fit()
print(model.summary())
